In [64]:
import re
import pandas as pd

In [65]:
# 读取文件
path = './收支分类表0318.xlsx'
out_path = './收支分类表0318_edited.xlsx'
df =pd.read_excel(path, sheet_name=0, header=1, engine='openpyxl')

In [66]:
# 填充空值，设定初始值
df.fillna('', inplace=True)
# 收入
df['租金'] = 0
df['管理费'] = 0
df['水费'] = 0
df['电费'] = 0
df['设备设施维护费'] = 0
df['停车费'] = 0
df['场地+球场+卫生费+杂费'] = 0
df['押金'] = 0
df['奖金/利息/退款'] = 0
df['还股东款'] = 0
df['税费'] = 0
# 支出
# df['租金.1'] = 0
# df['水费.1 '] = 0
# df['电费.1 '] = 0
# df['设备设施维护费.1'] = 0
df['工资'] = 0
df['工程款'] = 0
df['营销费'] = 0
df['运营费'] = 0
df['退款'] = 0    
df.head()

,日期,户名,摘要,收,支,Unnamed: 5,租金,管理费,水费,电费,...,税费,租金.1,水费.1,电费.1,设备设施维护费.1,工资,工程款,营销费,运营费,退款
0,2023-03-13,临保停车,收临保停车费1924，付手续费10.41,1913.59,,,0,0,0,0,...,0,,,,,0,0,0,0,0
1,2023-03-13,临保停车,收临保停车费19，付手续费0.11,18.89,,,0,0,0,0,...,0,,,,,0,0,0,0,0
2,2023-03-13,待报解预算收入-银税通,扣缴教育费附加\城市维护建设税\地方教育附加\增值税,,483.53,,0,0,0,0,...,0,,,,,0,0,0,0,0
3,2023-03-13,待报解预算收入-银税通,扣缴印花税,,5977.84,,0,0,0,0,...,0,,,,,0,0,0,0,0
4,2023-03-13,待报解预算收入-银税通,扣缴个人所得税,,2255.87,,0,0,0,0,...,0,,,,,0,0,0,0,0


In [67]:
# 检查数据
print(df.shape)
print(df.columns)
print(df.info())

(108, 26)
Index(['日期', '户名', '摘要', '收', '支', 'Unnamed: 5', '租金', '管理费', '水费', '电费',
       '设备设施维护费', '停车费', '场地+球场+卫生费+杂费', '押金', '奖金/利息/退款', '还股东款', '税费',
       '租金.1', '水费.1', '电费.1', '设备设施维护费.1', '工资', '工程款', '营销费', '运营费', '退款'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   日期            108 non-null    datetime64[ns]
 1   户名            108 non-null    object        
 2   摘要            108 non-null    object        
 3   收             108 non-null    object        
 4   支             108 non-null    object        
 5   Unnamed: 5    108 non-null    object        
 6   租金            108 non-null    int64         
 7   管理费           108 non-null    int64         
 8   水费            108 non-null    int64         
 9   电费            108 non-null    int64         
 10  设备设施维护费       108 non-null    int

In [68]:
# 处理收入项
for i, row in df.iterrows():
    c = row['摘要']
    if c != '合计' or c is not None:
        recv1 = re.findall(r'收.*?租金(\d+\.?\d*)', c)
        recv11 = re.search(r'收.*?租金$', c)
        recv21 = re.search(r'收.*?月.*?欠款', c)
        recv91 = re.findall(r'多交了(\d+\.?\d*)元*', c)
        if recv1:
            if len(recv1) > 1:
                rents = [float(r) for r in recv1]
                rent = sum(rents)
            else:
                rent = float(recv1[0])
            if recv91:
                if len(recv91) == 1:
                    extra = float(recv91[0])
                rent = rent + extra
            df.loc[i, '租金'] = rent
        if (not (recv1 and recv91)) and (recv11 or recv21):
            # print(i, c)
            rent = float(df.loc[i, '收'])
            df.loc[i, '租金'] = rent
            
        recv2 = re.findall(r'管理费(\d+\.?\d*)', c)
        recv12 = re.search(r'管理费|收.*?应缴费用', c)
        if recv2:
            if len(recv2) > 1:
                admins = [float(a) for a in recv2]
                admin = sum(admins)
            else:
                admin = float(recv2[0])
            df.loc[i, '管理费'] = admin
        if not recv2 and recv12:
            admin = float(df.loc[i, '收'])
            df.loc[i, '管理费'] = admin

        recv3 = re.findall(r'收.*?水费(\d+\.?\d*)', c)
        recv13 = re.search(r'收.*?水费', c)
        if recv3:
            if len(recv3) > 1:
                waters = [float(w) for w in recv3]
                water = sum(waters)
            else:
                water = float(recv3[0])
            df.loc[i, '水费'] = water
        if not recv3 and recv13:
            water = float(df.loc[i, '收'])
            df.loc[i, '水费'] = water

        recv4 = re.findall(r'收.*?电费(\d+\.?\d*)', c)
        recv14 = re.search(r'收.*?电费|收.*?电增容费', c)
        if recv4:
            if len(recv4):
                electrics = [float(e) for e in recv4]
                electric = sum(electrics)
            else:
                electric = float(recv4[0])
            df.loc[i, '电费'] = electric
        if not recv4 and recv14:
            electric = float(df.loc[i, '收'])
            df.loc[i, '电费'] = electric

        recv6 = re.search(r'停车费(\d+\.?\d*)', c)
        recv16 = re.search(r'月保停车费|收临保停车费', c)
        recv26 = re.search(r'收优惠停车券款', c)
        recv0 = re.search(r'手续费(\d+\.?\d*)', c)
        if recv6:
            fee = 0
            parking = float(recv6.group(1))
            if recv0:
                fee = float(recv0.group(1))
            df.loc[i, '停车费'] = parking - fee
        if not recv6 and (recv16 or recv26):
            parking = float(df.loc[i, '收'])
            df.loc[i, '停车费'] = parking

        recv5 = re.findall(r'设施维护\D?(\d+\.?\d*)', c)
        recv35 = re.findall(r'设备维护\D?(\d+\.?\d*)', c)
        recv45 = re.findall(r'设施设备\D?(\d+\.?\d*)', c)
        recv15 = re.search(r'设施维护费', c)
        recv25 = re.search(r'服务费(\d+\.?\d*)', c)
        if recv5:
            fee = 0
            if len(recv5):
                maintains = [float(m) for m in recv5]
                maintain = sum(maintains)
            else:
                maintain = float(recv5[0])
            if recv0:
                fee = float(recv0.group(1))
            if not recv6:
                df.loc[i, '设备设施维护费'] = maintain - fee
            else:
                df.loc[i, '设备设施维护费'] = maintain
        if recv35:
            fee = 0
            if len(recv35):
                maintains = [float(m) for m in recv35]
                maintain = sum(maintains)
            else:
                maintain = float(recv35[0])
            if recv0:
                fee = float(recv0.group(1))
            if not recv6:
                df.loc[i, '设备设施维护费'] = maintain - fee
            else:
                df.loc[i, '设备设施维护费'] = maintain
        if recv45:
            fee = 0
            if len(recv45):
                maintains = [float(m) for m in recv45]
                maintain = sum(maintains)
            else:
                maintain = float(recv45[0])
            if recv0:
                fee = float(recv0.group(1))
            if not recv6:
                df.loc[i, '设备设施维护费'] = maintain - fee
            else:
                df.loc[i, '设备设施维护费'] = maintain
        if recv25:
            maintain = float(recv25.group(1))
            df.loc[i, '设备设施维护费'] = maintain
        if not (recv5 or recv25 or recv35 or recv45) and recv15:
            maintain = float(df.loc[i, '收'])
            df.loc[i, '设备设施维护费'] = maintain

        recv7 = re.search(r'押金(\d+\.?\d*)', c)
        recv17 = re.search(r'押金|保证金|仓库款', c)
        recv27 = re.search(r'保证金(\d+\.?\d*)|保证金差额(\d+\.?\d*)', c)
        recv37 = re.search(r'水电周转金(\d+\.?\d*)', c)
        if recv7:
            guarantee = float(recv7.group(1))
            df.loc[i, '押金'] = guarantee
        if recv27:
            guarantee = float(recv27.group(1))
            df.loc[i, '押金'] = guarantee
        if recv37:
            guarantee = float(recv37.group(1))
            df.loc[i, '押金'] = guarantee
        if not (recv7 or recv27 or recv37) and recv17:
            guarantee = df.loc[i, '收']
            df.loc[i, '押金'] = guarantee

        recv8 = re.search(r'利息(\d+\.?\d*)', c)
        recv18 = re.search(r'利息|收结息|收.*?奖', c)
        if recv8:
            interest = float(recv8.group(1))
            df.loc[i, '奖金/利息/退款'] = interest
        if not recv8 and recv18:
            maintain = float(df.loc[i, '收'])
            df.loc[i, '奖金/利息/退款'] = maintain
        
        recv9 = re.search(r'场使用费(\d+\.?\d*)', c)
        recv19 = re.search(r'场地借用费(\d+\.?\d*)', c)
        recv29 = re.search(r'场地使用费|场使用费|场地借用费|稳岗返还款|产业发展专项资金|收社保退费|收场地服务费|收.*?场地租赁费', c)
        if recv9:
            space = float(recv9.group(1))
            df.loc[i, '场地+球场+卫生费+杂费'] = space
        if recv19:
            space = float(recv19.group(1))
            df.loc[i, '场地+球场+卫生费+杂费'] = space
        if not (recv9 or recv19) and recv29:
            space = float(df.loc[i, '收'])
            df.loc[i, '场地+球场+卫生费+杂费'] = space

In [69]:
# 处理支出项
for i, row in df.iterrows():
    c = row['摘要']
    if c != '合计' or c is not None:
        pay1 = re.search(r'电信费|汇划费|快递费|办公用品费|付工艺品款|认证审核费|运营费|打印费|乘车费|核酸检测费|桶装水款|体检费|垃圾清运费|绿化养护费|付律师费', c)
        pay11 = re.search(r'报销款|借支备用金|扣.*?邮电费|付.*?服务费|扣.*?月社保|扣.*?月住房公积金|付.*?保安巡逻费|付.*?保安服务加班费', c)
        pay21 = re.search(r'收回备用金', c)
        if pay1 or pay11:
            try:
                operation = float(df.loc[i, '支'])
                df.loc[i, '运营费'] = operation
            except Exception:
                print(f'运营费{pay1} of {pay11}')
        if pay21:
            operation = float(df.loc[i, '收'])
            df.loc[i, '运营费'] = -operation

        pay2 = re.search(r'工程款|工程费用|工程费|维保保养费|维护保养费|电梯配件|施工质保金|安全技术服务费|付修理费', c)
        if pay2:
            project = float(df.loc[i, '支'])
            df.loc[i, '工程费'] = project

        pay3 = re.search(r'制作费|设计费|点心款|茶歇款|付.*?礼品费|付活动.*?款', c)
        if pay3:
            marketing = float(df.loc[i, '支'])
            df.loc[i, '营销费'] = marketing

        pay4 = re.search(r'个人所得税|印花税|教育附加|增值税|城市维护建设税|企业所得税', c)
        if pay4:
            tax = float(df.loc[i, '支'])
            df.loc[i, '税费'] = tax

        pay5 = re.search(r'付.*?水费(\d+\.?\d*)', c)
        if pay5:
            try:
                water2 = float(pay5.group(1))
                df.loc[i, '水费.1'] = water2
            except Exception:
                print(f'水费：{pay5}')

        pay6 = re.search(r'付.*?电费(\d+\.?\d*)', c)
        pay16 = re.search(r'扣.*?供电局电费', c)
        if pay6:
            try:
                electric2 = float(pay6.group(1))
                df.loc[i, '电费.1'] = electric2
            except Exception:
                print(f'电费：{pay6}')
        if not pay6 and pay16:
            electric2 = float(df.loc[i, '支'])
            df.loc[i, '电费.1'] = electric2

        pay7 = re.search(r'退保证金|退水电周转金|退.*?保证金|退\w+费', c)
        if pay7:
            refund = float(df.loc[i, '支'])
            df.loc[i, '退款'] = refund

        pay8 = re.search(r'代发.*?工资', c)
        if pay8:
            salary = float(df.loc[i, '支'])
            df.loc[i, '工资'] = salary
        
        pay9 = re.search(r'还股东.*?本金', c)
        if pay9:
            shareholder = float(df.loc[i, '支'])
            df.loc[i, '还股东款'] = shareholder
            
        pay10 = re.search(r'付.*?月租金', c)
        if pay10:
            rent1 = float(df.loc[i, '支'])
            df.loc[i, '租金.1'] = rent1

In [70]:
# 格式化：替换0值，忽略错误，时间显示，列名
df.replace(0, '', inplace=True)
df['日期'] = pd.to_datetime(df['日期'], errors='coerce').dt.strftime('%Y/%#m/%#d')
df.rename(columns={'Unnamed: 5': ''}, inplace=True)

In [71]:
# 保存输出文件
df.to_excel(out_path, 'sheet1', index=False, startrow=1)

In [72]:
# 计算收支分类项的平衡，需要重新读取
df = pd.read_excel(out_path, sheet_name=0, header=1, engine='openpyxl')
# df['收支分类'] = df['租金'].astype(float) + df['管理费'].astype(float) + df['水费'].astype(float) + df['电费'].astype(float) + df['设备设施维护费'].astype(float) + df['停车费'].astype(float) + df['场地+球场+卫生费+杂费'].astype(float)+ df['押金'].astype(float) + df['利息'].astype(float) + df['还股东款'].astype(float) + df['税费'].astype(float) + df['租金.1'].astype(float) + df['水费.1'].astype(float) + df['电费.1'].astype(float) + df['工资'].astype(float) + df['工程费'].astype(float) + df['营销费'].astype(float) + df['运营费'].astype(float) + df['费用支出'].astype(float) + df['退款'].astype(float)
df['收支分类'] = df.iloc[:, 6:30].sum(axis=1)
df['收支小计'] = df[['收', '支']].sum(axis=1)
df['差额'] = abs(df['收支分类'] - df['收支小计'])

# 保留两位小数再比较
df.round({'收支分类': 2, '收支小计': 2})
df['平衡'] = df.apply(lambda x: '是' if abs(x['收支分类'] - x['收支小计']) < 0.01 else '否', axis=1)

# 检查是否有负数
df['平衡'] = df[df['平衡'] == '否'].apply(lambda x: '有负数' if x.loc['租金':'退款'].any() < 0 else '否', axis=1)

# 重新添加平衡值
df['平衡'].fillna('是', inplace=True)

In [73]:
df.head()

,日期,户名,摘要,收,支,Unnamed: 5,租金,管理费,水费,电费,...,工资,工程款,营销费,运营费,退款,工程费,收支分类,收支小计,差额,平衡
0,2023/3/13,临保停车,收临保停车费1924，付手续费10.41,1913.59,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1913.59,1913.59,0.0,是
1,2023/3/13,临保停车,收临保停车费19，付手续费0.11,18.89,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.89,18.89,0.0,是
2,2023/3/13,待报解预算收入-银税通,扣缴教育费附加\城市维护建设税\地方教育附加\增值税,NaN,483.53,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,483.53,483.53,0.0,是
3,2023/3/13,待报解预算收入-银税通,扣缴印花税,NaN,5977.84,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5977.84,5977.84,0.0,是
4,2023/3/13,待报解预算收入-银税通,扣缴个人所得税,NaN,2255.87,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2255.87,2255.87,0.0,是


In [74]:
# 再次保存
imbalance = df['平衡'].str.count('否').sum()
df.to_excel(out_path, 'sheet1', index=False, startrow=1)
print(f'已经生成分类完成的Excel文件{out_path}，其中有{imbalance}处不平衡，请检查')

已经生成分类完成的Excel文件./收支分类表0318_edited.xlsx，其中有3处不平衡，请检查
